In [4]:
# IMPORTS
import numpy as np
import pandas as pd

import re
import requests

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

In [6]:
## HW Q1

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

ipo_filings = pd.read_html(response.text)
filings = ipo_filings[0]
filings

/tmp/ipykernel_23537/1669464139.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_filings = pd.read_html(response.text)


,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"May 3, 2024",TBN,Tamboran Resources Corporation,-,-
1,"Apr 29, 2024",HWEC,"HW Electro Co., Ltd.",$3.00,3750000
2,"Apr 29, 2024",DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
3,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000
4,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,-
...,...,...,...,...,...
321,"Jan 21, 2020",GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000
322,"Jan 21, 2020",UTXO,"UTXO Acquisition, Inc.",$10.00,5000000
323,"Dec 9, 2019",LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000
324,"Oct 4, 2019",ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000


In [7]:
filings['Filing Date'] = pd.to_datetime(filings['Filing Date'], format='mixed')
filings[filings['Shares Offered'].astype(str).str.find('-') >= 0]

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,2024-05-03,TBN,Tamboran Resources Corporation,-,-
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,-
5,2024-04-22,DRJT,Derun Group Inc,$5.00,-
7,2024-04-16,JLJT,Jialiang Holdings Ltd,$5.00,-
8,2024-04-15,GAUZ,Gauzy Ltd.,-,-
...,...,...,...,...,...
301,2021-03-02,NSAL,"Navios South American Logistics, Inc.",-,-
315,2021-01-19,DECI,"Decipher Biosciences, Inc.",-,-
317,2020-09-08,GMWX,"GameWorks, Inc.",-,-
318,2020-09-04,SCIT,Sancai Holding Group Ltd.,-,-


In [8]:
def extract_prices(input_string):
    """
    :params input_strings:
        1. replace '-' with NaN, if no price is specified
        2. return the original price, if only one number provided, e.g. $4.00
        3. return average of 2 price ranages, e.g. $8.00-$10.00
    """
    if input_string == '-': 
        return np.NaN
    else: 
        price_list = input_string.replace('$', '').split('-')
        float_price = [float(price) for price in price_list]
        return sum(float_price) / len(float_price)

filings['Avg_price'] = filings['Price Range'].apply(extract_prices)
filings

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price
0,2024-05-03,TBN,Tamboran Resources Corporation,-,-,NaN
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000,3.00
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000,10.00
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000,10.00
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,-,NaN
...,...,...,...,...,...,...
321,2020-01-21,GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000,9.00
322,2020-01-21,UTXO,"UTXO Acquisition, Inc.",$10.00,5000000,10.00
323,2019-12-09,LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000,9.00
324,2019-10-04,ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000,4.00


In [9]:
filings['Shares Offered'] = pd.to_numeric(filings['Shares Offered'], errors='coerce')
filings['Shares_offered_value'] = filings['Shares Offered'] * filings['Avg_price']
filings_2023Fri = filings[
    (filings['Filing Date'].dt.dayofweek == 4) & (filings['Filing Date'].dt.year == 2023)]
filings_2023Fri.shape

(32, 7)

In [10]:
filings_2023Fri['Shares_offered_value'].sum()

285700000.0

# 1) [Code Snippet 1] IPOs data from Web

* learn dataframes "stacking" (UNION ALL)
* learn about type casting
* generate new columns (simple and with function)


In [11]:
import pandas as pd
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

/tmp/ipykernel_23537/4213396001.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_dfs = pd.read_html(response.text)


In [12]:
ipos_2023 = ipo_dfs[0]
ipos_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      154 non-null    object
 1   Symbol        154 non-null    object
 2   Company Name  154 non-null    object
 3   IPO Price     154 non-null    object
 4   Current       154 non-null    object
 5   Return        154 non-null    object
dtypes: object(6)
memory usage: 7.3+ KB


In [13]:
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

/tmp/ipykernel_23537/1522156903.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_dfs = pd.read_html(response.text)


In [14]:
ipos_2024 = ipo_dfs[0]
ipos_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      65 non-null     object
 1   Symbol        65 non-null     object
 2   Company Name  65 non-null     object
 3   IPO Price     65 non-null     object
 4   Current       65 non-null     object
 5   Return        65 non-null     object
dtypes: object(6)
memory usage: 3.2+ KB


In [15]:
# "stacking dataframes"
# pd.concat() is used to concatenate the DataFrames vertically.
# The ignore_index=True parameter ensures that the resulting DataFrame has a new index,
# ignoring the original indices of the input DataFrames.
# The stacked_df now contains the concatenated DataFrame.
stacked_ipos_df = pd.concat([ipos_2024, ipos_2023], ignore_index=True)

In [16]:
stacked_ipos_df.head(1)

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"May 8, 2024",NNE,Nano Nuclear Energy Inc.,$4.00,$5.19,29.75%


In [17]:
# Need to convert everything to a proper type (date, str, int, float, etc.)
stacked_ipos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219 entries, 0 to 218
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      219 non-null    object
 1   Symbol        219 non-null    object
 2   Company Name  219 non-null    object
 3   IPO Price     219 non-null    object
 4   Current       219 non-null    object
 5   Return        219 non-null    object
dtypes: object(6)
memory usage: 10.4+ KB


In [19]:
# convert to datetime
stacked_ipos_df['IPO Date'] = pd.to_datetime(stacked_ipos_df['IPO Date'], format='mixed')

In [45]:
ipo_tickers = stacked_ipos_df[stacked_ipos_df['IPO Date'] < '2024-03-01']['Symbol']
# ipo_tickers.drop('RYZB', inplace=True)
len(ipo_tickers)

185

In [44]:
for ticker in ipo_tickers: 
    if ticker == 'RYZB': 
        continue
    else: 
        yf.download(tickers = ticker,
                     period = "max",
                     interval = "1d")

34     SMXT
35     VHAI
36     DYCQ
37     CHRO
38     UMAC
       ... 
214     QSG
215    CVKD
216    SKWD
217    ISRL
218    MGOL
Name: Symbol, Length: 185, dtype: object

In [51]:
price_df = yf.download(tickers='SMXT', period='max', interval='1d')
price_df.head(1)['Adj Close']

[*********************100%%**********************]  1 of 1 completed


Date
2024-02-27    8.0
Name: Adj Close, dtype: float64

In [ ]:
price_df['Adj Close']

In [ ]:
def calculate(df: pd.DataFrame): 
    """
    :params df: stock price table retrieved from Yahoo Finance,
    with columns of Open, High, Low, Close, Adj Close, Volume. 
    """
    start_price = df.head(1)['Adj Close']
    